In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd

In [3]:
import lib
import plotting

In [4]:
from multiprocessing import Pool
import pickle
from collections import defaultdict

In [5]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score, calinski_harabaz_score
from sklearn.decomposition import PCA

In [6]:
from configparser import ConfigParser
cfg = ConfigParser()
cfg.read("config.cfg")

['config.cfg']

In [7]:
from tqdm._tqdm_notebook import tqdm_notebook as tqdm # we manually import the notebook submodule as the normal one struggles with jupyterlab
tqdm.pandas() # this enables us to use progress_apply instead of apply

In [8]:
results_prefix = "[WIKIDATA_BOTEXT]"

sequence_file = os.path.join(cfg.get("directory", "exchange"), "[wikidata]004_sequences.p")
labels_file = os.path.join(cfg.get("directory", "exchange"), "[wikidata]004_labels.p")
users_file = os.path.join(cfg.get("directory", "exchange"), "[wikidata]004_users.p")

In [9]:
df = pd.read_pickle(sequence_file)
labels = pd.read_pickle(labels_file)['label'].values
df.head()

,length,sequence,user_id
0,1511,"[REVERT_MANUAL, BREAK, CLAIM_CREATE, BREAK, CL...",1
1,9,"[ENTITY_CREATE, BREAK, SITELINK_ADD, BREAK, ME...",1000036
2,7,"[SITELINK_ADD, BREAK, SITELINK_ADD, BREAK, SIT...",1000078
3,9,"[ENTITY_CREATE, BREAK, SITELINK_REMOVE, SITELI...",100008
4,44,"[SITELINK_ADD, BREAK, SITELINK_ADD, BREAK, SIT...",100012


In [10]:
def calc_dist(sequence):
    dist, pivot = lib.calc_distribution(labels, sequence)
    return np.array(dist)

with Pool(cfg.getint("core", "num_cores")) as processor_pool:
    df['stat_dist'] = pd.Series(processor_pool.imap(calc_dist, tqdm(df['sequence'])))

In [11]:
df.head()

,length,sequence,user_id,stat_dist
0,1511,"[REVERT_MANUAL, BREAK, CLAIM_CREATE, BREAK, CL...",1,"[0.022260285362644863, 0.01999793118304165, 0...."
1,9,"[ENTITY_CREATE, BREAK, SITELINK_ADD, BREAK, ME...",1000036,"[0.020070153072094386, 0.02007015307209438, 0...."
2,7,"[SITELINK_ADD, BREAK, SITELINK_ADD, BREAK, SIT...",1000078,"[0.01947397679104996, 0.019473976791049966, 0...."
3,9,"[ENTITY_CREATE, BREAK, SITELINK_REMOVE, SITELI...",100008,"[0.02042483183496783, 0.020424831834967825, 0...."
4,44,"[SITELINK_ADD, BREAK, SITELINK_ADD, BREAK, SIT...",100012,"[0.01973684210526316, 0.019736842105263157, 0...."


In [12]:
kernel_range = np.arange(cfg.getint("kmeans", "clusters_min"), cfg.getint("kmeans", "clusters_max") + 1)
kernel_range

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10])

In [13]:
vectors = np.array([np.array(x) for x in df['stat_dist'].values]) # ensure we have the data in numpy format
pca = PCA(n_components=3)
plot_vectors = pca.fit_transform(vectors)
print(pca.explained_variance_ratio_.cumsum())

[0.28125296 0.4497747  0.57094982]


In [34]:
centroids = {}    
centroids_file = os.path.join(cfg.get("directory", "exchange"), results_prefix + "001_centroids.p")
load_centroids = cfg.getboolean("kmeans", "load_centroids") and os.path.isfile(centroids_file)
for num_centroids in tqdm(kernel_range):    
    kmeans = None

    print("Num Centroids: {n}".format(n=num_centroids))
    if load_centroids:
        with open(centroids_file, "rb") as dump_file:
            centroids = pickle.load(dump_file)
        kmeans = KMeans(n_clusters=num_centroids, init=centroids[num_centroids])
    else:
        kmeans = KMeans(n_clusters=num_centroids)
        
    kmeans.fit_predict(vectors)
    
    lbl = kmeans.labels_
    centroids[num_centroids] = kmeans.cluster_centers_

    
    silhouette_avg = silhouette_score(vectors, lbl)#
    print("SILHOUETTE", silhouette_avg)
    calinski_score = calinski_harabaz_score(vectors, lbl)
    print("CALINSKI", calinski_score)
    sample_silhouette_values = silhouette_samples(vectors, lbl)
    
    store_path = os.path.join(cfg.get("directory", "results"), results_prefix + "kmeans[{n}]".format(n=num_centroids))
    cluster_lbl = "kmeans_{n}".format(n=num_centroids)
    df[cluster_lbl] = lbl
    plotting.k_means(plot_vectors, num_centroids, lbl, sample_silhouette_values, silhouette_avg, store_path)
    
if cfg.getboolean("kmeans", "store_centroids"):
    with open(centroids_file, "wb") as dump_file:
        pickle.dump(centroids, dump_file)

Num Centroids: 2


/home/pkasper/python/Python-3.7.1/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:971: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


SILHOUETTE 0.24386984466175524
CALINSKI 23333.643977691416
data/results/[WIKIDATA]kmeans[2]_silhouette.png
data/results/[WIKIDATA]kmeans[2]_silhouette.pdf
data/results/[WIKIDATA]kmeans[2]_clusters.png
data/results/[WIKIDATA]kmeans[2]_clusters.pdf
data/results/[WIKIDATA]kmeans[2]_clusters_planar.png
data/results/[WIKIDATA]kmeans[2]_clusters_planar.pdf
data/results/[WIKIDATA]kmeans[2]_population.png
data/results/[WIKIDATA]kmeans[2]_population.pdf
Num Centroids: 3


/home/pkasper/python/Python-3.7.1/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:971: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


SILHOUETTE 0.29026215981061654
CALINSKI 22633.81826219971
data/results/[WIKIDATA]kmeans[3]_silhouette.png
data/results/[WIKIDATA]kmeans[3]_silhouette.pdf
data/results/[WIKIDATA]kmeans[3]_clusters.png
data/results/[WIKIDATA]kmeans[3]_clusters.pdf
data/results/[WIKIDATA]kmeans[3]_clusters_planar.png
data/results/[WIKIDATA]kmeans[3]_clusters_planar.pdf
data/results/[WIKIDATA]kmeans[3]_population.png
data/results/[WIKIDATA]kmeans[3]_population.pdf
Num Centroids: 4


...

SILHOUETTE 0.38909583497882905
CALINSKI 22845.047431541298
data/results/[WIKIDATA]kmeans[8]_silhouette.png
data/results/[WIKIDATA]kmeans[8]_silhouette.pdf
data/results/[WIKIDATA]kmeans[8]_clusters.png
data/results/[WIKIDATA]kmeans[8]_clusters.pdf
data/results/[WIKIDATA]kmeans[8]_clusters_planar.png
data/results/[WIKIDATA]kmeans[8]_clusters_planar.pdf
data/results/[WIKIDATA]kmeans[8]_population.png
data/results/[WIKIDATA]kmeans[8]_population.pdf
Num Centroids: 9


/home/pkasper/python/Python-3.7.1/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:971: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


SILHOUETTE 0.3344967246708248
CALINSKI 22450.464803944174
data/results/[WIKIDATA]kmeans[9]_silhouette.png
data/results/[WIKIDATA]kmeans[9]_silhouette.pdf
data/results/[WIKIDATA]kmeans[9]_clusters.png
data/results/[WIKIDATA]kmeans[9]_clusters.pdf
data/results/[WIKIDATA]kmeans[9]_clusters_planar.png
data/results/[WIKIDATA]kmeans[9]_clusters_planar.pdf
data/results/[WIKIDATA]kmeans[9]_population.png
data/results/[WIKIDATA]kmeans[9]_population.pdf
Num Centroids: 10


/home/pkasper/python/Python-3.7.1/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:971: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


SILHOUETTE 0.3551217829206689
CALINSKI 22674.948195454668
data/results/[WIKIDATA]kmeans[10]_silhouette.png
data/results/[WIKIDATA]kmeans[10]_silhouette.pdf
data/results/[WIKIDATA]kmeans[10]_clusters.png
data/results/[WIKIDATA]kmeans[10]_clusters.pdf
data/results/[WIKIDATA]kmeans[10]_clusters_planar.png
data/results/[WIKIDATA]kmeans[10]_clusters_planar.pdf
data/results/[WIKIDATA]kmeans[10]_population.png
data/results/[WIKIDATA]kmeans[10]_population.pdf


In [35]:
df.to_pickle(os.path.join(cfg.get("directory", "exchange"), results_prefix + "001_clustering.p"))

In [36]:
labels_filtered = sorted(["BREAK", "DESCRIPTION_UPDATE", "CLAIM_CREATE", "REFERENCE_ADD", "ENTITY_OVERRIDE", "ENTITY_REDIRECT", "MERGE", "DESCRIPTION_ADD", "LABEL_ADD", "QUALIFIER_ADD", "CLAIM_UPDATE", "SITELINK_ADD", "SITELINK_UPDATE", ])
labels_filtered

['BREAK',
 'CLAIM_CREATE',
 'CLAIM_UPDATE',
 'DESCRIPTION_ADD',
 'DESCRIPTION_UPDATE',
 'ENTITY_OVERRIDE',
 'ENTITY_REDIRECT',
 'LABEL_ADD',
 'MERGE',
 'QUALIFIER_ADD',
 'REFERENCE_ADD',
 'SITELINK_ADD',
 'SITELINK_UPDATE']

In [40]:
for num_centroids in [7]:#kernel_range:
    member_count = defaultdict(int)
    cluster = defaultdict(list)
    cluster_lbl = "kmeans_{n}".format(n=num_centroids)
    
    for i, user in df.iterrows():
        cluster_id = user[cluster_lbl]
        member_count[cluster_id] += 1
        action_df = pd.DataFrame({"from": user['sequence']})
        action_df["to"] = action_df["from"].shift(periods=-1)
        cluster[cluster_id].append(action_df)
         
    for cluster_index in cluster:
        cluster_df = pd.concat(cluster[cluster_index])

        transition_count_pivot = lib.stretch_pivot(pd.crosstab(cluster_df['from'], cluster_df['to']), labels)
        unnormalized = transition_count_pivot.transpose().sum().transpose()
        pivot = lib.stretch_pivot(pd.crosstab(cluster_df['from'], cluster_df['to'], normalize="index"), labels)
        
        store_path = os.path.join(cfg.get("directory", "results"), results_prefix + "clusters[{n}_{i}]".format(n=num_centroids, i=cluster_index))
        plotting.transition_matrix((50, 60), unnormalized, pivot, transition_count_pivot, cluster_index, store_path)
        
        
        transition_count_pivot_reduced = lib.stretch_pivot(transition_count_pivot, labels_filtered)
        unnormalized_reduced = transition_count_pivot_reduced.transpose().sum().transpose()
        pivot_reduced = lib.stretch_pivot(pd.crosstab(cluster_df['from'], cluster_df['to'], normalize="index"), labels_filtered)
        
        store_path = os.path.join(cfg.get("directory", "results"), results_prefix + "clusters[{n}_{i}]reduced".format(n=num_centroids, i=cluster_index))
        plotting.transition_matrix((20, 25),unnormalized_reduced, pivot_reduced, transition_count_pivot_reduced, cluster_index, store_path)

data/results/[WIKIDATA]clusters[7_4].png
data/results/[WIKIDATA]clusters[7_4].pdf
data/results/[WIKIDATA]clusters[7_4]reduced.png
data/results/[WIKIDATA]clusters[7_4]reduced.pdf
data/results/[WIKIDATA]clusters[7_0].png
data/results/[WIKIDATA]clusters[7_0].pdf
data/results/[WIKIDATA]clusters[7_0]reduced.png
data/results/[WIKIDATA]clusters[7_0]reduced.pdf
data/results/[WIKIDATA]clusters[7_1].png
data/results/[WIKIDATA]clusters[7_1].pdf
data/results/[WIKIDATA]clusters[7_1]reduced.png
data/results/[WIKIDATA]clusters[7_1]reduced.pdf
data/results/[WIKIDATA]clusters[7_3].png
data/results/[WIKIDATA]clusters[7_3].pdf
data/results/[WIKIDATA]clusters[7_3]reduced.png
data/results/[WIKIDATA]clusters[7_3]reduced.pdf
data/results/[WIKIDATA]clusters[7_2].png
data/results/[WIKIDATA]clusters[7_2].pdf
data/results/[WIKIDATA]clusters[7_2]reduced.png
data/results/[WIKIDATA]clusters[7_2]reduced.pdf
data/results/[WIKIDATA]clusters[7_5].png
data/results/[WIKIDATA]clusters[7_5].pdf
data/results/[WIKIDATA]clust